In [1]:
import numpy as np
import cv2
import timm
import tqdm

import torch
from torchvision import datasets, transforms
from torch.utils import data

from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix
from PIL import Image

from matplotlib import pyplot as plt


np.random.seed(42)
NUM_CLASSES = 9
BATCH_SIZE = 32
device = torch.device("cuda")
PATH_TO_TEST_DATASET = "data/CRC-VAL-HE-7K/"


def cv2_loader(path: str):
    return Image.fromarray(cv2.imread(path, -1)[:, :, ::-1])


def cv2_jpeg80_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 80
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])

def cv2_jpeg60_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 60
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])


def cv2_jpeg40_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 40
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])


def cv2_jpeg20_damage_loader(path: str):
    img = cv2.imread(path, -1)
    quality = 20
    _, img = cv2.imencode('test.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, quality))
    img = cv2.imdecode(img, cv2.IMREAD_UNCHANGED)
    return Image.fromarray(img[:, :, ::-1])


/home/user/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/user/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-07-19 19:16:37.852925: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2024-07-19 19:16:37.852940: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from model import DeepCMorph

model = DeepCMorph(num_classes=NUM_CLASSES)
# Loading model weights corresponding to the TCGA Pan Cancer dataset
# Possible dataset values:  TCGA, TCGA_REGULARIZED, CRC, COMBINED
model.load_weights(dataset="CRC")

model.to(device)
model.eval()

Model loaded, unexpected keys: []


DeepCMorph(
  (dropout): Dropout(p=0.0, inplace=False)
  (model_preprocessing): DeepCMorphSegmentationModule(
    (encoder): EfficientNet(
      (features): Module(
        (0): Conv2dNormActivation(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): Module(
          (0): Module(
            (block): Module(
              (0): Conv2dNormActivation(
                (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
                (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
                (2): SiLU(inplace=True)
              )
              (1): SqueezeExcitation(
                (avgpool): AdaptiveAvgPool2d(output_size=1)
                (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
               

In [3]:
def evaluate(model, test_dataloader):
    print("Running Evaluation...")

    targets_array = []
    predictions_array = []

    with torch.no_grad():

        test_iter = iter(test_dataloader)
        for j in tqdm.tqdm(range(len(test_dataloader))):

            image, labels = next(test_iter)
            image = image.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)


            predictions = model(image)
            
            _, predictions = torch.max(predictions.data, 1)

            predictions = predictions.detach().cpu().numpy()
            targets = labels.detach().cpu().numpy()

            for k in range(targets.shape[0]):

                target = targets[k]
                predicted = predictions[k]

                targets_array.append(target)
                predictions_array.append(predicted)

        print("Accuracy: " + str(accuracy_score(targets_array, predictions_array)))
        print("Balanced Accuracy: " + str(balanced_accuracy_score(targets_array, predictions_array)))
        

        print(classification_report(targets_array, predictions_array))
        print(confusion_matrix(targets_array, predictions_array))
        
        
        return predictions_array

In [4]:
test_transforms = transforms.Compose([transforms.ToTensor()])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_no_jpeg_aug = evaluate(model, test_dataloader)

Running Evaluation...


  0%|                                                                                                  | 0/225 [00:00<?, ?it/s]/home/user/anaconda3/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 1.72 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,
100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [02:29<00:00,  1.51it/s]

Accuracy: 0.96991643454039
Balanced Accuracy: 0.9559323884341416
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1338
           1       1.00      1.00      1.00       847
           2       0.90      1.00      0.95       339
           3       0.98      0.99      0.99       634
           4       0.99      0.99      0.99      1035
           5       0.83      0.95      0.89       592
           6       0.99      0.99      0.99       741
           7       0.97      0.72      0.83       421
           8       0.97      0.99      0.98      1233

    accuracy                           0.97      7180
   macro avg       0.96      0.96      0.96      7180
weighted avg       0.97      0.97      0.97      7180

[[1309    0    0    3    3   23    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  338    1    0    0    0    0    0]
 [   0    0    1  626    0    0    0    0    7]
 [   0    0    0    0 1022   10    0  

In [5]:
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg20_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg20_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [02:30<00:00,  1.50it/s]

Accuracy: 0.881058495821727
Balanced Accuracy: 0.8416215715468965
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1338
           1       0.90      1.00      0.94       847
           2       1.00      0.74      0.85       339
           3       0.87      1.00      0.93       634
           4       0.79      0.96      0.86      1035
           5       0.88      0.50      0.64       592
           6       0.94      0.97      0.95       741
           7       0.46      0.59      0.52       421
           8       1.00      0.83      0.90      1233

    accuracy                           0.88      7180
   macro avg       0.87      0.84      0.84      7180
weighted avg       0.90      0.88      0.88      7180

[[1317   10    0    0    6    4    0    0    1]
 [   0  847    0    0    0    0    0    0    0]
 [   0   49  251    0   25    7    0    7    0]
 [   0    0    0  634    0    0    0    0    0]
 [   8   27    0    0  991    3    1 

In [6]:
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg40_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg40_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [02:29<00:00,  1.51it/s]

Accuracy: 0.950974930362117
Balanced Accuracy: 0.9376232329147309
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1338
           1       0.99      1.00      1.00       847
           2       1.00      0.96      0.98       339
           3       0.92      1.00      0.96       634
           4       0.96      0.99      0.97      1035
           5       0.90      0.80      0.85       592
           6       0.92      0.99      0.96       741
           7       0.74      0.78      0.76       421
           8       0.99      0.93      0.96      1233

    accuracy                           0.95      7180
   macro avg       0.94      0.94      0.94      7180
weighted avg       0.95      0.95      0.95      7180

[[1315    4    0    1   11    7    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  327    8    1    3    0    0    0]
 [   0    0    0  634    0    0    0    0    0]
 [   5    2    0    0 1022    1    1 

In [7]:
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg60_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg60_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [02:27<00:00,  1.53it/s]

Accuracy: 0.9622562674094708
Balanced Accuracy: 0.9510598327835322
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1338
           1       1.00      1.00      1.00       847
           2       1.00      0.99      0.99       339
           3       0.94      1.00      0.97       634
           4       0.97      0.99      0.98      1035
           5       0.88      0.89      0.89       592
           6       0.94      0.99      0.97       741
           7       0.82      0.77      0.80       421
           8       0.99      0.95      0.97      1233

    accuracy                           0.96      7180
   macro avg       0.95      0.95      0.95      7180
weighted avg       0.96      0.96      0.96      7180

[[1313    0    0    3   11   11    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  335    4    0    0    0    0    0]
 [   0    0    0  634    0    0    0    0    0]
 [   1    0    0    0 1023    4    1

In [8]:
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_jpeg80_damage_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_jpeg80_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [02:28<00:00,  1.52it/s]

Accuracy: 0.9681058495821727
Balanced Accuracy: 0.9570684244263621
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1338
           1       1.00      1.00      1.00       847
           2       0.99      1.00      0.99       339
           3       0.95      1.00      0.97       634
           4       0.99      0.99      0.99      1035
           5       0.85      0.94      0.90       592
           6       0.95      1.00      0.97       741
           7       0.91      0.75      0.82       421
           8       0.99      0.97      0.98      1233

    accuracy                           0.97      7180
   macro avg       0.96      0.96      0.96      7180
weighted avg       0.97      0.97      0.97      7180

[[1306    0    0    5    7   20    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  338    1    0    0    0    0    0]
 [   0    0    0  634    0    0    0    0    0]
 [   0    0    0    0 1024    6    1